In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append('../')
sys.path.append('../../')
sys.path.append('../../../')

In [ ]:
import math as m
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import scripts.lammps.iotools as lmpio
import sklearn

In [ ]:
full_clusterlist = [
    "sp3a", "sp3b", "sp3c", "sp4a", "sp4b", "6A", "sp5a", "sp5b", "sp5c",
     "6Z", "7K", "7T_a", "7T_s", "8A", "8B", "8K", "9A", "9B", "9K", "10A", "10B", "10K", "10W", "11A", "11B",
    "11C", "11E", "11F", "11W", "12A", "12B", "12D",
    "12E", "12K", "13A", "13B", "13K", "FCC", "HCP", "BCC_9"
]

excluded_clusterlist = [
    "10W", "11A", "11B", "11W",
    "12K", "13A", "13K", "10K",
#  "FCC", "HCP",
]

included_clusterlist = [clust for clust in full_clusterlist if clust not in excluded_clusterlist]
print(len(full_clusterlist),  len(included_clusterlist))

In [ ]:
import matplotlib as mpl
hfont = {'family':'sans-serif', 'size': 8}
colors = {}
for c in full_clusterlist:
    colors[c] = '#2D63D1'
colors['FCC'] = colors['FCC/HCP'] = '#2D63D1'
colors['8A'] = '#ff75ac'
colors['sp5c'] = '#714f99'
colors['HCP'] = colors['sp5c/8A'] = '#A1322C'
colors['6A'] = colors['HCP']

labels = {}
labels['8A'] = 'SD'
labels['sp5c'] = 'PB'
labels['6A'] = 'OH'
labels['FCC'] = 'FCC'
labels['HCP'] = 'HCP'

In [ ]:
def convolve(df, n=20, sigma=1, gaussian=False):
    df_ = {}
    for col in df.columns:
        # df_[col] = df[col]
        if not gaussian:
            df_[col] = np.convolve(df[col], np.ones(n), 'same') / n
        else:
            x = np.arange(-n, n+1) #* 180/len(hist)
            kernel = np.exp(-0.5*(x/sigma)**2)
            # print(kernel/kernel.sum())
            df_[col] = np.convolve(df[col], kernel, mode='same')/kernel.sum()
    return df_

# Population level TCC plots

# SI: all clusters as function of number of solidlike bonds

In [ ]:
clusts = {}
press = 13.4
nbins = 13
legend = True

clusters_to_plot = included_clusterlist 
clusters_to_plot = ['sp5c', '8A', 'FCC', 'HCP']
# mode = 'clusters-per-particle'
mode = 'particles-in-cluster'


if len(clusters_to_plot) == 32:
    fig, axs = plt.subplots(8, 4, sharex=True, sharey='row', figsize=(7,8))
    axs = np.concatenate(axs)
    plt.subplots_adjust(hspace=0.1)
    plt.subplots_adjust(wspace=0.1)
else:
    fig, ax = plt.subplots(figsize=(2.81, 1.9), sharex=True)
    plt.subplots_adjust(hspace=0.05)
    axs = len(clusters_to_plot) * [ax,]

for data in ['sim', 'exp']:
    for ic, clusts['A'] in enumerate(clusters_to_plot):
        if data == 'sim':
            if not os.path.exists(f"../../results/conversion/betap{press:.2f}/trial2/dump/tcc-particle-correlation-wolde-{clusts['A']}-{mode}.log"):
                continue

        df_ = None
        for traj in [2,]:
            # read data
            if data == 'sim':
                dir1 = f'../../results/conversion/betap{press:.2f}/trial{traj}/dump'
            if data == 'exp':
                dir1 = f'../../results/experiments/dump'
            df = pd.read_csv(f"{dir1}/tcc-particle-correlation-wolde-{clusts['A']}-{mode}.log")
            df = df.fillna(0)

            # compute weighted averages
            for i in range(nbins):
                df[f'{i}'] = df[f'number_of_particles_{i}'] * df[f'clusters_per_particle_{i}']

            # average over different trajectories
            sum = df.sum(axis=0)
            if df_ is None:
                df_ = sum
            else:
                df_ += sum
        
        # gather into array
        x = np.arange(nbins)
        y = np.zeros(nbins)
        for i in range(nbins):
            y[i] = df_[f'{i}'] / df_[f'number_of_particles_{i}']

        # plot
        if len(clusters_to_plot) > 5:
            label = data.upper() if ic == 0 else None
            marker = '-o' if data == 'sim' else '-^'
            color = '#2D63D1' if data == 'sim' else '#A1322C'
        else:
            # label = f"{clusts['A']:5s} ({data.upper()})"
            label = data.upper() if ic == 0 else None
            marker = '-o' if data == 'sim' else '-^'
            color = colors[clusts['A']]
            # marker += 'o' if ic == 0 else '^'
         #
        if data == 'sim' and len(clusters_to_plot) > 5:
            label_ = labels[clusts['A']] if clusts['A'] in labels else clusts['A']
            y_text = 0.8 if clusts['A'] in ['6A', 'sp5c', '8A', '9B', '9K', 'BCC_9', 'sp4a', 'sp3a'] else 0.2
            weight = 'bold' if clusts['A'] in ['8A', '6A', 'sp5c', 'FCC', 'HCP'] else None
            axs[ic].text(0.1, y_text, label_, transform=axs[ic].transAxes, weight=weight, size=8)
        axs[ic].fill_between([5.5, 7.5], 0, 10, color='#e2e2e2')
        axs[ic].plot(x, y, marker, c=color, label=label, markersize=3.5)
        
    # style
    if len(axs) == 0:
        # for tick in axs[0].xaxis.get_major_ticks():
        #     tick.label.set_fontsize(18)
        # for tick in axs[0].yaxis.get_major_ticks():
        #     tick.label.set_fontsize(18)
        axs[0].minorticks_on()
    for ax in axs:
        ax.set_xlim(-0.2, 12.2)
        ymax = 1 if mode == 'particles-in-cluster' else 10
        ax.set_ylim(0, ymax)
        if len(clusters_to_plot) > 5:
            ax.set_xticks([6, 12])

if len(clusters_to_plot) > 5:
    fig.legend(bbox_to_anchor=(0.43, 1.0, 0.2, 0.2), loc="lower left", ncol=2, mode="expand", fontsize=8)
                # mode="expand", borderaxespad=0, ncol=3)
    # fig.supylabel(r'$N_{CL} ~/~ N$', **hfont)
    fig.supylabel('Average number of clusters per particle', **hfont)
    fig.supxlabel('Number of solidlike bonds', **hfont)
else:
    if legend:
        leg = fig.legend(bbox_to_anchor=(1.2, 0.75, 0.2, 0.2), loc="upper right")
        for lh in leg.legendHandles:
            lh.set_color('black')
    if mode == 'clusters-per-particle':
        plt.ylabel('Average number of clusters per particle', **hfont)
    else:
        plt.ylabel('P', **hfont)
    plt.xlabel('Number of solidlike bonds', **hfont) #
    plt.xticks([0, 2, 4, 6, 8, 10, 12], **hfont)
    plt.yticks([0, 0.25, 0.5, 0.75, 1], **hfont)
fig.tight_layout()

if len(clusters_to_plot) > 5:
    plt.savefig(f"../../results/paper-figs/SI_all-clusters-vs-solidlike.pdf", bbox_inches='tight', dpi=600)
else:
    plt.savefig(f"../../results/paper-figs/sp5c-8a-vs-solidlike.png", bbox_inches='tight', dpi=600, )

## Fig 2a: sp5c, 8A and FCC/HCP during nucleation as function of time

In [ ]:
for data in ['coli', 'lj', 'yukawa']:
    # 'hermes', 'fioru',
    print(data)
    figsize = (2.81, 1.9) #(2.81, 4.02) if data == 'coli' else (2.81, 1.9)

    fig, ax = plt.subplots(figsize=figsize)
    ic = 0

    plt.subplots_adjust(hspace=0.1)
    plt.subplots_adjust(wspace=0.1)
    import matplotlib as mpl

    hfont = {'family':'sans-serif', 'size': 8}
    mpl.rc('font',family='sans-serif', size=8)

    colors = {}
    for c in full_clusterlist:
        colors[c] = 'C0'
    colors['FCC'] = '#2D63D1'
    colors['8A'] = '#ff75ac'
    colors['sp5c'] = '#714f99'
    colors['HCP'] = '#A1322C'
    ks8 = f"{['sp5c']}-{['8A']}"
    k8F = f"{['8A']}-{['FCC', 'HCP']}"
    ksF = f"{['sp5c']}-{['FCC', 'HCP']}"

    labels = {}
    all_labels = True
    labels['FCC'] = 'FCC' if all_labels else None #'Face-centered cubic (FCC)'
    labels['HCP'] = 'HCP' if all_labels else None#'Hexagonal close-packed (HCP)'
    labels['sp5c'] = 'PB' if all_labels else None #Pentagonal Bipyramid (PB)
    labels['8A'] = 'SD' if all_labels else None #Siamese Dodecahedron (SD)
    labels['6A'] = 'OH'
    labels[ks8] = 'PB or SD'
    labels[k8F] = 'SD but not FCC or HCP'
    labels[ksF] = 'sp5c-solid'

    w, sigma = 8, 8
    conversion_averages = {}
    clusts = {}
    key = 'particles_in_cluster' #'number_of_clusters' # # # # # #
    subtract_fluid = False
    normalize = True
    plot_a_or_b = True
    plot_only_average = False
    alpha = 0.1
    skip_fcc = False
    # clusters_to_plot = ['FCC', 'HCP'] #
    clusters_to_plot = ['sp5c', '8A', 'FCC', 'HCP']
    # clusters_to_plot = included_clusterlist

    averages = {}
    for c in clusters_to_plot + [ks8,]:
        averages[c] = 0

    trajmax = 5 if data == 'coli' else 3
    for traj in range(trajmax): #[1,2,3,5,]: #[2,3,6,9,11]:
        # read data
        if data == 'coli':
            press = 13.4
            d_in = f'../../results/conversion/betap{press:.2f}/trial{[2,3,6,9,11][traj]}'
        elif data == 'fioru':
            d_in = f'../../results/fioru/RHO_0.77700/RUN_00{[2,3,4,5,][traj]}'
        elif data == 'hermes':
            d_in = f'../../results/hermes/rate_1.022_0{[1,2,3,5,][traj]}/'
        elif data == 'yukawa':
            d_in = f'../../results/yukawa/{[8769, 8890, 8956][traj]}'
        elif data == 'lj':
            d_in = f'../../results/lj/{[6922, 7867, 8205, 8355][traj]}'
            # i = 
            # logdir = d_out = d_in = d0 + f'{i}/'



        df, df_ = {}, {}
        for c in clusters_to_plot:
            df[c] = pd.read_csv(f"{d_in}/tcc-basic-{c}.log")
            df_[c] = convolve(df[c], n=w, sigma=sigma)
        for c in [ks8,]:
            df[c] = pd.read_csv(f"{d_in}/tcc-particle-cluster-competition-{c}.log")
            df_[c] = convolve(df[c], n=w, sigma=sigma)

        # set time
        if data == 'coli':
            t0 = 180
            mask = np.arange(t0-160, t0+41)
            mask = mask[mask>0]
            time = 10/m.sqrt(40) * (np.arange(len(df[clusters_to_plot[0]]['frame'])))
            time -= -300+time[mask].max()
        elif data == 'fioru':
            t0 = 100
            mask = np.arange(t0-33, t0+10)
            mask = mask[mask>0]
            time = 0.0004486 * 1e5 / m.sqrt(40) * (np.arange(len(df[clusters_to_plot[0]]['frame'])))
            time -= -300 + time[mask].max()
        elif data == 'hermes':
            t0 = 150
            mask = np.arange(200)
            mask = mask[mask>0]
            time = np.arange(len(df[clusters_to_plot[0]]['frame'])) / 2.169960e+00
            time -= -90 + time[mask].max()
        elif data == 'yukawa':
            t0 = 100
            mask = np.arange(200)
            mask = mask[mask>0]
            time = np.arange(len(df[clusters_to_plot[0]]['frame']))
            time -= -200 + time[mask].max()
        elif data == 'lj':
            t0 = 100
            mask = np.arange(200)
            mask = mask[mask>0]
            time = np.arange(len(df[clusters_to_plot[0]]['frame']))
            time -= -200 + time[mask].max()
        
        for c in clusters_to_plot:
            y = df_[c][key][mask]
            if normalize:
                y /= df_[clusters_to_plot[0]]['number_of_particles'][mask]
            if not plot_only_average:
                ax.plot(time[mask], y, alpha=alpha, c=colors[c])
            averages[c] += y

        if plot_a_or_b:
            if key == 'particles_in_cluster':
                y = df_[ks8]['particles_in_A_or_B'][mask] #- df_[ks8]['particles_in_B'][mask]
            else:
                y = df_['sp5c'][key][mask] + df_['8A'][key][mask]
                y /= 2

            if normalize:
                y /= df_['sp5c']['number_of_particles'][mask]
            if not plot_only_average:
                ax.plot(time[mask], y, ':', alpha=alpha, c='k')
            averages[ks8] += y

    # plot populations averaged over different trajectories
    for k in averages.keys():
        # print(k)
        y = averages[k] / trajmax
        if k == ks8:
            if plot_a_or_b:
                ax.plot(time[mask], y, ':', label=labels[k], c='k')
        elif k == k8F:
            ax.plot(time[mask], y, ':', label=labels[k], c=colors['8A'])
        elif k == ksF:
            pass
            # axs[ic].plot(time[mask], y, ':', label=labels[k], c=colors['FCC'])
        else:
            ax.plot(time[mask], y, label=labels[k], c=colors[k])
        # print(k, y[:len(y)//2].mean())

    # print(averages['FCC'][:len(y)//2].mean() / (averages['FCC'][:len(y)//2].mean() + averages['HCP'][:len(y)//2].mean()))
    # print(averages['FCC'][len(y)//2:].mean() / (averages['FCC'][len(y)//2:].mean() + averages['HCP'][len(y)//2:].mean()))

    # style
    ax.minorticks_on()

    T = 300 if data == 'coli' else 200
    ax.set_xlim(time[mask].max()-T, time[mask].max())
    # ax.set_xticks(np.arange(10)*10)
    if subtract_fluid:
        ax.set_ylim(-0.2, 0.2)
    else:
        pass
        # ax.set_ylim(0, 1)
        # ax.set_ylim(0, 0.15)

    if subtract_fluid:
        fig.supylabel(r'$(N_{CL} - \bar{N}_{CL}) ~/~ N$', x=0.04, y=0.6, **hfont)
    else:
        # fig.supylabel(r'$N_{CL} ~/~ N$', **hfont, x=0.1,) #
        # plt.ylabel('Average number of clusters per particle', ) #y=0.6, x=0.04
        ylabel = 'P' #'P (particle in cluster)' if data == 'coli' else 'P'
        plt.ylabel(ylabel, )
    plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', ) # **hfont #x=0.55, y=0.06,
    # fig.legend(loc='upper left', bbox_to_anchor=(0.234, 0.75, 0.2, 0.2), ncol=2)

    # change order of legend
    handles, labels = plt.gca().get_legend_handles_labels()
    print(labels)
    order = [4, 1, 0, 2, 3]
    yleg = 0.2 if data == 'coli' else 0.23
    # if data == 'coli':
    #     fig.legend(
    #         [handles[idx] for idx in order], [labels[idx] for idx in order],
    #         loc='lower left', bbox_to_anchor=(0.21, yleg, 0.1, 0.2)) #
    fig.tight_layout()
    plt.savefig(f"../../results/paper-figs/{data}-sp5c-8a-fcc.svg", dpi=600)

# Fig 4b: internal angles of 8A and sp5c clusters

In [ ]:
import matplotlib as mpl
clust = '8A'
fig, ax = plt.subplots(figsize=(5.35, 2.95))
hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)
dfs = {}

for clust in ['sp5c', '8A']:
    for ring in [True]:
        distributions = {}

        sums = {}
        for label in ['pre', 'post']:
            sums[label] = None

        for traj in [2,3,6,9,11]:# 6, 9, 11]: #3, 
            # read data
            data = f'betap13.40/trial{traj}'
            if ring:
                for label in ['pre', 'post']:
                    dfs[label] = pd.read_csv(f"../../results/conversion/{data}/dump/tcc-{clust}-ring-angles-{label}_.log")
            else:
                for label in ['pre', 'post']:
                    dfs[label] = pd.read_csv(f"../../results/conversion/{data}/dump/tcc-{clust}-not-ring-angles-{label}.log")
            
            for label in ['pre', 'post']:
                if sums[label] is None:
                    sums[label] = dfs[label].sum().values[1:]
                else:
                    sums[label] += dfs[label].sum().values[1:]
                distributions[label] = (sums[label])

        for ig, label in enumerate(['pre', 'post']):
            hist = distributions[label]
            print(hist.sum())
            hist = hist/hist.sum()
            # w = 1
            # hist_ = np.convolve(hist, np.ones(w), mode='same')/w
            
            # smooth
            n, sigma = 2, 1
            x = np.arange(-n, n+1) * 180/len(hist)
            kernel = np.exp(-0.5*(x/sigma)**2)
            print(kernel/kernel.sum())
            hist_ = np.convolve(hist, kernel, mode='same')/kernel.sum()

            # plot
            c = colors[[clust, clust][ig]]
            ls = ['-', '--'][ig] # if ring else ':'
            label_ = ['Pre-conversion', 'Post-conversion'][ig]
            ax.plot(180*np.arange(len(hist_))/len(hist_), hist_, linestyle=ls, label=label_, c=c)

plt.xlabel(r'$\theta$')
plt.ylabel(r'$P(\theta)$')
# plt.xticks([i*np.pi/6 for i in range(2,5)], labels=[r'$\pi/3$', r'$\pi/2$', r'$2\pi/3$'])
# [r'$\pi/6$', r'$\pi/3$', r'$\pi/2$', r'$2\pi/3$', r'$5\pi/6$']
# plt.xticks([50, 70, 90, 110, 130])
plt.xlim(0, 180)
plt.xticks([0, 30, 60, 90, 120, 150, 180])

plt.ylim(0, 0.05)
# plt.legend() #title='Overlap with FCC/HCP'
plt.tight_layout()
plt.savefig('../../results/paper-figs/8a-internal-angle.svg', dpi=600, bbox_inches='tight')

# SI: all clusters over time during nucleation

In [ ]:
fig, axs = plt.subplots(8, 4, sharex=True, sharey='row', figsize=(7,8)) #(8.27,11.69)
axs = np.concatenate(axs)

plt.subplots_adjust(hspace=0.1)
plt.subplots_adjust(wspace=0.1)

hfont = {'family':'sans-serif', 'size': 8}
colors = {}
for c in full_clusterlist:
    colors[c] = 'C0'
colors['FCC'] = colors['FCC/HCP'] = '#2D63D1'
colors['8A'] = '#ff75ac'
colors['sp5c'] = '#714f99'
colors['6A'] = colors['HCP'] = colors['sp5c/8A'] = '#A1322C'

labels = {}
labels['8A'] = 'SD'
labels['sp5c'] = 'PB'
labels['6A'] = 'OH'

w = 20
conversion_averages = {}
clusts = {}
prekey = 'particles_in_cluster' #'p_num'
subtract_fluid = False
normalize = True
plot_a_or_b = True
skip_fcc = False
press = 13.4

# clusters_to_plot = np.array(['BCC_9', '8B', '7T_a', '7T_s', '7K', '9K', 'sp5b', '6Z', '11F', 'sp3b',
#  'FCC','6A', 'sp3c', 'sp5a', 'sp4b', '9B', 'sp3a' ,'sp4a', '8A', '11E', 'sp5c',
#  '12E', 'HCP', '10B', '11C', '12D', '8K' ,'9A', '12B', '13B', '10K'])

clusters_to_plot = included_clusterlist
mean_pops = []

for ic, clusts['A'] in enumerate(clusters_to_plot):
    averages = {}
    for ik, k in enumerate([prekey]):
        averages[k] = 0

    for traj in [2,3,6,9,11]:
        # read data
        data = f'betap{press:.2f}/trial{traj}'
        df = pd.read_csv(f"../../results/conversion/{data}/tcc-basic-{clusts['A']}.log")
        df_ = convolve(df, n=w)

        # set time
        t0 = 180
        mask = np.arange(t0-160, t0+61)
        mask = mask[mask>0]
        time = 10/m.sqrt(40) * np.arange(len(df))
        time -= -300+time[mask].max()
                
        # plot populations
        y = df_[prekey][mask]
        colorkey = clusts['A']

        if subtract_fluid:
            y -= y[:len(y)//2].mean()
        if normalize:
            y /= df_['number_of_particles'][mask]
        # save for averages
        averages[prekey] += y
        
        # plot
        axs[ic].plot(time[mask], y, alpha=0.2, c=colors[colorkey])
    
    # plot populations averaged over different trajectories
    colorkey = clusts['A']
    y = averages[prekey] / 5
    if subtract_fluid:
        y -= y[:len(y)//2].mean()
    mean_pops.append(y.mean())
    axs[ic].plot(time[mask], y, label='number_of_clusters', c=colors[colorkey])
    label = labels[clusts['A']] if clusts['A'] in labels else clusts['A']
    weight = 'bold' if clusts['A'] in ['8A', '6A', 'sp5c', 'FCC', 'HCP'] else None
    axs[ic].text(0.1, 0.1, label, transform=axs[ic].transAxes, weight=weight, size=8)
    axs[ic].fill_between([time[160], time[200]], 0, 10, color='#e2e2e2')
    print(time[160], time[200])

    # style
    if len(axs) == 0:
        for tick in axs[0].xaxis.get_major_ticks():
            tick.label.set_fontsize(8)
        for tick in axs[0].yaxis.get_major_ticks():
            tick.label.set_fontsize(8)
        axs[0].minorticks_on()

for iax, ax in enumerate(axs):
    ax.set_xlim(time[mask].max()-300, time[mask].max())
    ax.set_ylim(0, 1)

if subtract_fluid:
    fig.supylabel(r'$(N_{CL} - \bar{N}_{CL}) ~/~ N$', **hfont)
else:
    fig.supylabel('P (particle in cluster)', **hfont)
    # fig.supylabel(r'$N_{CL} ~/~ N$', **hfont)
fig.supxlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', **hfont)
fig.tight_layout()
plt.savefig(f"../../results/paper-figs/SI_all_clusters_time.pdf", dpi=300)

sorted = np.argsort(mean_pops)
print(np.array(clusters_to_plot)[sorted])
print(np.array(mean_pops)[sorted])

# SI: Conversion of sp5c->8A and 8A -> FCC/HCP

In [ ]:
hfont = {'family':'sans-serif', 'size':8}

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
mpl.rc('font',family='sans-serif', size=8)
colorPalette = {
    'FCC': '#2D63D1',
    'sp5c': '#ff75ac',
    '8A': '#714f99',
    'HCP': '#A1322C'
}

n = 1
w = 20
plots = True
conversion_averages = {}

# clusts = {'A': 'sp5c'}
clusts = {'A': '8A'}
# clusts = {'A': 'FCC'}

for clusts['B'] in ['FCC', 'HCP']: #['FCC', 'HCP']: #
    averages = {}
    for i, k in enumerate(['AB', 'BA',]):
        averages[k] = 0
    averages['sB'] = 0
        
    for traj in [2,3,6,9,11]: #
        # read data
        data = f'betap13.40/trial{traj}' # -max_frames-5
        df = pd.read_csv(f"../../results/conversion/{data}/cluster-conversion-{clusts['A']}-{clusts['B']}.log")
        df_ = convolve(df, n=w,)

        df_basic = pd.read_csv(f"../../results/conversion/{data}/tcc-basic-{clusts['A']}.log")
        df_basic_ = convolve(df_basic, n=w,)

        # set time
        t0 = 180
        mask = np.arange(t0-160, t0+41)
        time = 10/m.sqrt(40) * (np.arange(len(df['frame'])))
        time -= -300+time[:-n][mask].max()
                
        # plot conversion
        enum = ['AB', 'BA',] if clusts['A'] == 'sp5c' else ['AB']
        for i, k in enumerate(enum): #'BA',
            # if i == 1 and (clusts['A'] == '8A'):
            #     continue
            a, b = k
            x = time[:-n][mask]
            y = df_['conversions'+k][n:][mask] / df_[f'number_of_{a}_clusters'][:-n][mask]
            averages[k] += y
            if plots:
                ax.plot(x, y, alpha=0.2, c=colorPalette[clusts[b]])
        
        y = 1 - df_[f'number_of_{a}_clusters'][:-n][mask] / df_basic_['number_of_clusters'][:-n-1][mask]
        averages['sB'] += y

    labels_ = [f"{labels[clusts['A']]} -> {labels[clusts['B']]}", f"{labels[clusts['B']]} -> {labels[clusts['A']]}"]
    
    enum = ['AB', 'BA',] if clusts['A'] == 'sp5c' else ['AB']
    for i, k in enumerate(enum): #'sB',
        # if i == 1 and (clusts['A'] == '8A'):
        #     continue
        a, b = k
        y = averages[k] / 5
        if k != 'sB':
            conversion_averages[clusts['B']] = y[len(y)//2:].mean()
        ls = ':' if k == 'sB' else '-'
        if plots:
            ax.plot(x, y, linestyle=ls, label=labels_[i], c=colorPalette[clusts[b]])
        print(y[:125].mean())

# if clusts['B'] == 'FCC':
#     print(conversion_averages['FCC'] / (conversion_averages['FCC'] + conversion_averages['HCP']))

# style
if plots:
    ax.minorticks_on()

    ax.set_xlim(x.max()-300, x.max())
    ax.set_ylabel('P (conversion)', **hfont)
    if clusts['B'] in ['FCC', 'HCP']:
        ax.set_ylim(0, 0.14)
        # ax.set_yticks([0, 0.05, 0.1, 0.15, 0.2])
    else:
        ax.set_ylim(0, 0.05)
        pass
        # ax.set_ylim(0, 0.12)
        # ax.set_ylim(0, 0.08)
        # ax.set_yticks([0, 0.02, 0.04, 0.06, 0.08])
    ax.set_xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', **hfont)

    # ax.set_yscale('log')

if plots:
    fig.tight_layout()
    plt.legend(loc='best') #lower left
    plt.savefig(f"../../results/paper-figs/conversion-{clusts['A']}-{clusts['B']}.svg", dpi=300, bbox_inches='tight')
    plt.show()

## SI: Subcell correlation between 8A and crystalline particles

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
df = pd.read_csv('../../results/coli/Plots/SI_Fig2/cluster_corr.dat', delim_whitespace=True, names=['frame', 'corr'])
df_ = convolve(df, )

t0 = 184
mask = np.arange(t0-160, t0+41 + 500)
mask = mask[mask>0]
time = 10/m.sqrt(40) * (np.arange(len(df['frame'])))
time -= time[mask].min()
# time -= -300-1*100 + time[mask][:201].max()

plt.plot(time[mask], df_['corr'][mask])

plt.xlim(0, 600)
plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', )
plt.ylabel(r'$r(n_{SD}, n_{X})$')
plt.yticks([-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1])
plt.ylim(-1, 1)
plt.savefig('../../results/paper-figs/8a-x-correlation-time.svg', dpi=300, bbox_inches='tight')
plt.show()

fig, ax = plt.subplots(1, figsize=(4, 3))
# df = pd.read_csv('../../results/coli/Plots/SI_Fig2/mixedMany.dat', delim_whitespace=True, names=['cell', '8a', 'X', '1', '2'])
df = pd.read_csv('../../results/coli/Plots/SI_Fig2/cells_t3.dat', delim_whitespace=True, names=['cell', '8a', 'X', '1', '2'])


plt.axhline(df.mean()['X'], c='#e2e2e2')
plt.axvline(df.mean()['8a'], c='#e2e2e2')
plt.scatter(df['8a'], df['X'], s=5, c=colors['FCC'], zorder=1e9)

plt.xlim(0.6, 1)
plt.ylim(0, 0.4)
plt.ylabel(r'$n_X$')
plt.xlabel(r'$n_{SD}$')
plt.savefig('../../results/paper-figs/8a-x-correlation.svg', dpi=300, bbox_inches='tight')
plt.show()

# SI: How fcc/hcp-like are 8A clusters

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
mpl.rc('font',family='sans-serif', size=8)
clust = '8A'
df_ = None
trajs = [2,3,6,9,11]

# time
w = 20
t0 = 180
mask = np.arange(t0-160, t0+41)
mask = mask[mask>0]
time = 10/m.sqrt(40) * np.arange(250)
time -= -300+time[mask].max()

# groups = [[0,1,2], [3,4,5], [6,7,8]]
groups = [[0], [1,2,3,4,5,6,], [7,8]]
labels_ = [r'$0$', r'$\geq 1,\leq 6$', r'$>6$']

for traj in [2,3,6,9,11]: #:
    # read data
    data = f'betap13.40/trial{traj}'
    df = pd.read_csv(f"../../results/conversion/{data}/dump/tcc-surface-clusters-{clust}.log")

    if df_ is None:
        df_ = df
    else:
        df_ += df

    df = convolve(df, n=w)

    for ig, group in enumerate(groups):
        sum = 0
        for i in group:
            sum += df[f'overlap_{i}']
        total = 0
        for i in range(9):
            total += df[f'overlap_{i}']

        c = colors[['sp5c', '8A', 'FCC'][ig]]
        plt.plot(time[mask], sum[mask] / (total[mask]), c=c, alpha=0.2)

w = 20
df_ = convolve(df_, n=w)

# plot averages
for ig, group in enumerate(groups):
    sum = 0
    for i in group:
        sum += df_[f'overlap_{i}']
    total = 0
    for i in range(9):
        total += df_[f'overlap_{i}']

    c = colors[['sp5c', '8A', 'FCC'][ig]]
    plt.plot(time[mask], sum[mask] / (total[mask]), label=labels_[ig], c=c)

plt.xlim(time[mask].max()-300, time[mask].max())
plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', **hfont)
plt.ylabel('Fraction of SD clusters', **hfont)
plt.ylim(0, 1)
plt.legend(title='Overlap with FCC/HCP')
fig.tight_layout()
plt.savefig('../../results/paper-figs/8a-overlap.svg', dpi=600, bbox_inches='tight')

# 8a ring vs spindle vs shifted

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
mpl.rc('font',family='sans-serif', size=8)

d0 = '../../results/conversion/betap13.40/'
df_ = None
w = 20

colors['shifted'] = colors['8A']
colors['rings'] = colors['sp5c']
colors['spindles'] = colors['6A']

# time
t0 = 180
mask = np.arange(t0-160, t0+41)
mask = mask[mask>0]
time = 10/m.sqrt(40) * np.arange(250)
time -= -300+time[mask].max()

for i in [2,3,6,9,11]:
    # read data
    logdir = f'../../results/conversion/betap13.40/trial{i}'
    df = pd.read_csv(f'{logdir}/dump/tcc-8a-overlap.log')
    if df_ is None:
        df_ = df
    else:
        df_ += df

    # plot
    df = convolve(df, n=w,)
    for label in ['rings', 'spindles', 'shifted']:
        y = df[f'solid_{label}'] / (df[f'number_{label}'])
        plt.plot(time[mask], y[mask], alpha=0.1, c=colors[label])

    # y = df[f'solid'] / df[f'number_of_particles']
    # plt.plot(time[mask], y[mask], alpha=0.1, c=colors['FCC'])

# plot averages
df_ = convolve(df_, n=w,)
for label in ['rings', 'spindles', 'shifted']:
    y = df_[f'solid_{label}'] / df_[f'number_{label}']
    label_ = label[:-1] if label.endswith('s') else label
    plt.plot(time[mask], y[mask], label=label_, c=colors[label])

# y = df_[f'solid'] / df_[f'number_of_particles']
# plt.plot(time[mask], y[mask], label='total', c=colors['FCC'])

plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', )
plt.xlim(time[mask].max()-300, time[mask].max())
plt.ylim(0, 1)
plt.ylabel('P (particle in FCC or HCP)', **hfont)
plt.legend()
fig.tight_layout()
plt.savefig('../../results/paper-figs/SI_8a-overlap-parts.svg', dpi=300, bbox_inches='tight')
plt.show()

## SI: amount of FCC/HCP clusters instead of populations

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
mpl.rc('font',family='sans-serif', size=8)

d0 = '../../results/conversion/betap13.40/'
w = 20

clusters_to_plot = ['FCC', 'HCP']

# time
t0 = 180
mask = np.arange(t0-160, t0+41)
mask = mask[mask>0]
time = 10/m.sqrt(40) * np.arange(250)
time -= -300+time[mask].max()

df, df_ = {}, {}
av_df = {}
key = 'number_of_clusters'
for c in clusters_to_plot:
    av_df[c] = 0
av_df['N'] = 0

for i in [2,3,6,9,11]:
    # read data
    logdir = f'../../results/conversion/betap13.40/trial{i}'

    for c in clusters_to_plot:
        df[c] = pd.read_csv(f"{logdir}/tcc-basic-{c}.log")
        df_[c] = convolve(df[c], n=w)
        av_df[c] += df_[c][key]
    av_df['N'] += df_[c]['number_of_particles']

    # plot
    plt.plot(time[mask], df_['FCC'][key][mask] / (df_['FCC'][key][mask] + df_['HCP'][key][mask]), alpha=0.2, c=colors['FCC'])

# plot averages
plt.plot(time[mask], av_df['FCC'][mask] / (av_df['FCC'][mask] + av_df['HCP'][mask]), c=colors['FCC'])

plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', )
plt.xlim(time[mask].max()-300, time[mask].max())
plt.ylim(0, 1)
plt.ylabel(r'$n_{fcc} ~/~ (n_{fcc} + n_{hcp})$', **hfont)
# plt.legend()
fig.tight_layout()
plt.savefig('../../results/paper-figs/SI-fcc-hcp-num-clusters.svg', dpi=300, bbox_inches='tight')
plt.show()

## SI: FCC/HCP with different classification schemes

In [ ]:
fig, ax = plt.subplots(1, figsize=(4, 3))
mpl.rc('font',family='sans-serif', size=8)

df = pd.read_csv('../../results/coli/Plots/SI_Fig4/all_no.dat', delim_whitespace=True, header=None).values

t0 = 184
mask = np.arange(t0-160, t0+41)
mask = mask[mask>0]
time = 10 / m.sqrt(40) * (np.arange(df.shape[0]))
time -= time[mask].min()

for i in range(12):
    y = df[:, 1+2*i]
    y = np.convolve(y, np.ones(20)/20, mode='same')
    plt.plot(time, y) #[24:274]


plt.xlim(0, 300)
plt.xlabel(r'$t~/~\sqrt{\beta m \sigma^2}$', )
plt.ylim(0, 1)
plt.ylabel(r'$n_{fcc} ~/~ (n_{fcc} + n_{hcp})$', **hfont)
# plt.legend()
fig.tight_layout()
plt.savefig('../../results/paper-figs/SI-fcc-hcp-different-methods.svg', dpi=300, bbox_inches='tight')
plt.show()

## SI: More on correlation between 8A and solid bonds

In [ ]:
d0 = '../../results/conversion/betap13.40/'
num_bins = 12
counts = np.zeros(num_bins)
counts_8a = np.zeros(num_bins)

for i in [2,]: #3,6,9,11
    print(i)
    logdir = f'../../results/conversion/betap13.40/trial{i}'
    for frame_number in range(125):
        with open(f'{logdir}/dump/{frame_number}/clusters.pickle', 'rb') as file:
            cluster_table, _ = pickle.load(file)
        with open(f'{logdir}/dump-bops/{frame_number}/solid-bonds.pickle', 'rb') as file:
            solid_bonds = pickle.load(file)

        # 8a
        for i in range(num_bins):
            mask_i = (cluster_table['8A'] == i)
            counts_8a[i] += solid_bonds[mask_i].mean() / 125

In [ ]:
d0 = '../../results/coli/Plots/SI_Fig3'
dfs = {}
dfs['all'] = pd.read_csv(f'{d0}/bop02000.dat', delim_whitespace=True, names=['q4', 'q6', 'w4', 'w6'])
for label in ['base', 'spindle', 'fifth']:
    dfs[label] = pd.read_csv(f'{d0}/q602000{label}.dat', delim_whitespace=True, names=['q6_bin', 'count'])
    bins = dfs[label]['q6_bin']
    count = dfs[label]['count']
    plt.plot(bins, count/count.sum(), label=label)
# plt.hist(dfs['all']['q6'], bins=50)
plt.xlim(0.1, 0.4)
plt.legend()
plt.show()

## Fig 2a: Mutual information on conversion to fcc/hcp

In [ ]:
from sklearn.feature_selection import mutual_info_classif

num_bins = 13
d0 = '../../results/conversion/betap13.40'
plots = True
clusters_to_plot = ['6A', 'sp5c', ]
clusters_to_plot = full_clusterlist #included_clusterlist
mi_totals = np.zeros(len(clusters_to_plot))
mi_polys = np.zeros(len(clusters_to_plot))

clustB = 'FCC'

for ic, clustA in enumerate(clusters_to_plot): 
    print(ic, clustA)

    # combine data of all trajectories
    for filter in [False]: #[False, True]:
        df_ = None
        for i in [2, 3, 6, 9, 11]:
            d1 = f'{d0}/trial{i}'
            if filter:
                df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}-filtered.log') # 
            else:
                df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}.log') # 
            df = df[:125]
            sum = df.sum(axis=0)
            if df_ is None:
                df_ = sum
            else:
                df_ += sum

        # initialize arrays
        X_total = np.empty(0)
        y_total = np.empty(0)
        X_poly = np.empty(0)
        y_poly = np.empty(0)
        conversions_ = np.nan*np.zeros(num_bins)
        source_ = np.nan*np.zeros(num_bins)
        y_fcc = np.nan*np.zeros(num_bins)
        y_hcp = np.nan*np.zeros(num_bins)
        y_neither = np.nan*np.zeros(num_bins)

        for j in range(num_bins):
            # number of particles part of clustA
            source = round(df_[f'number_of_particles_{j}'])

            # number of conversions to fcc/hcp
            conversions = round(df_[f'converted_{j}'])
            conversions_neither = conversions - df_[f'FCC_{j}'] - df_[f'HCP_{j}']
            conversions_fcc = round(df_[f'FCC_{j}'] + 0.5 * conversions_neither)
            conversions_hcp = round(df_[f'HCP_{j}'] + 0.5 * conversions_neither)

            # collect data in format convenient for mutual_info_classif
            X_total = np.concatenate([X_total, j*np.ones(source)])
            y_total = np.concatenate([y_total, np.ones(conversions)])
            y_total = np.concatenate([y_total, np.zeros(source-conversions)])

            # collect data in format convenient for mutual_info_classif
            X_poly = np.concatenate([X_poly, j*np.ones(conversions_fcc)])
            X_poly = np.concatenate([X_poly, j*np.ones(conversions_hcp)])
            y_poly = np.concatenate([y_poly, np.ones(conversions_fcc)])
            y_poly = np.concatenate([y_poly, np.zeros(conversions_hcp)])

            divisor = source if clusters_to_plot == ['8A'] else conversions #source
            conversions_[j] = conversions / source if source > 0 else np.nan
            source_[j] = source
            if divisor > 0:
                y_fcc[j] = conversions_fcc / divisor
                y_hcp[j] = conversions_hcp / divisor

    # compute mutual information
    mi_totals[ic] = mutual_info_classif(X_total.reshape(-1, 1), y_total, discrete_features=True)[0] if len(X_total)>0 else np.nan
    mi_polys[ic] = mutual_info_classif(X_poly.reshape(-1, 1), y_poly, discrete_features=True)[0] if len(X_poly)>0 else np.nan

    print(mi_totals[ic])

print('Total')
print(np.array(clusters_to_plot)[np.argsort(mi_totals)][::-1])
print(np.sort(mi_totals)[::-1] /mi_totals[0]) #

print('Poly')
print(np.array(clusters_to_plot)[np.argsort(mi_polys)][::-1])
print(np.sort((mi_polys))[::-1]/ np.sort((mi_polys))[-1] )

In [ ]:
import matplotlib as mpl
hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)

labels = {}
labels['BCC_9'] = 'BCC9'
labels['8A'] = 'SD'
labels['sp5c'] = 'PB'
labels['6A'] = 'OH'
# clusters_to_plot = included_clusterlist
# plt.scatter(mi_totals, mi_polys)
fig, ax = plt.subplots(figsize=(3, 3.5))

for i, c in enumerate(clusters_to_plot):
    if c == 'sp4c':
        continue
    x = mi_totals[i]
    y = mi_polys[i]
    
    plt.scatter(x, y, c='C0', s=10)
    label = labels[c] if c in labels else c
    if x + y > 1e-3:
        if c in ['12D', '11E', '9B', '11C']:
            # dx = {'8A': -1.2e-4, '6A': -1.2e-4, 'sp5c': 3e-5}[c]
            dx = 0e-5 if c == '12D' else 5e-5
            dy = -1.5e-4 if c == '12D' else -1e-5
            dy += 5e-5 if c in ['9B', '11C'] else 0
            dy += 5e-5 if c in ['11C'] else 0
            ax.annotate(c,
                xy=(x, y), xycoords='data',
                xytext=(x+dx, y+dy), textcoords='data',
                arrowprops=dict(arrowstyle="-",
                                connectionstyle="arc3", shrinkA=0.2, shrinkB=0.2),
                )
        else:
            if c in ['8A', '6A', 'sp5c']:
                plt.text(x+3e-5, y-2e-5, label, weight='bold')
            else:
                plt.text(x+3e-5, y-2e-5, label, )
                 #/np.sort((mi_polys))[-1]
    else:
        print(c)

        
# plt.xlabel("Logistic 'correlation' on crystallization")
# plt.ylabel("Logistic 'correlation' on polymorph selection")
plt.xlabel("MI on crystallization " + r'$\left(\times 10^{-3}\right)$')
plt.ylabel("MI on polymorph selection " + r'$\left(\times 10^{-3}\right)$')
plt.xlim(0, 1e-3)
plt.ylim(0, 3e-3)
plt.xticks(2e-4*np.arange(6), [f'{x:.1f}' for x in 2e-1*np.arange(6)])
plt.yticks(5e-4*np.arange(7), [f'{x:.1f}' for x in 5e-1*np.arange(7)])
# plt.ylim(-0.3, 0.3)
# plt.axhline(0, c='k')
# plt.axvline(0, c='k')
plt.tight_layout()
plt.savefig('../../results/paper-figs/mi.svg', dpi=600, bbox_inches='tight', pad_inches=0.1) #

## Fig 2c/d: polymorph selection by bipyramids

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.85))
# ax2 = ax.twinx()
hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)
colors['6A'] = colors['HCP']

num_bins = 13
d0 = '../../results/conversion/betap13.40'
clustB = 'FCC'

for ic, clustA in enumerate(['6A', 'sp5c',]):
    # combine data of all trajectories
    df_ = None
    for i in [2, 3, 6, 9, 11]:
        d1 = f'{d0}/trial{i}'
        # df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}-max-frames-2.log') # 
        df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}.log') # 
        df = df[:125]
        sum = df.sum(axis=0)
        if df_ is None:
            df_ = sum
        else:
            df_ += sum

    # initialize arrays
    conversions_ = np.nan*np.zeros(num_bins)
    source_ = np.nan*np.zeros(num_bins)

    for j in range(num_bins):
        # number of particles part of clustA
        source = round(df_[f'number_of_particles_{j}'])
        conversions = round(df_[f'converted_{j}'])
        converted_ = round(df_[f'FCC_{j}'])
        divisor = round(df_[f'HCP_{j}'])+ round(df_[f'FCC_{j}']) #conversions
        conversions_[j] = converted_ / divisor if divisor > 0 else np.nan
        source_[j] = source
    ax.plot(conversions_[:6], marker='o', c=colors[clustA])
    # ax2.plot(source_[:6]/source_[:6].sum(), marker='o', color=colors['HCP'])
    # ax2.set_yscale('log')
    # ax2.set_ylabel('Frequency')

    ax.set_ylabel(r'$P_{fcc} ~/~ (P_{fcc} + P_{hcp})$')
    # ax.set_ylabel(r'$P_{fcc} ~/~ P_{hcp}$')
    # ax.set_ylabel(r'$P ~(\rightarrow fcc) ~/~ (P ~(\rightarrow fcc) + P ~(\rightarrow hcp))$')
    ax.set_xlabel(f'Clusters per particle')
    # ax.set_ylim(2, 3.5)
    ax.set_yticks([0.5, 0.6, 0.7, 0.8], labels=['0.50', '0.60', '0.70', '0.80'])
    # ax.set_yticks([0.68, 0.7, 0.72, 0.74, 0.76, 0.78])
    ax.set_xticks(range(6))
    ax.minorticks_on()
    ax.tick_params(axis='x', which='minor', bottom=False)
        
plt.tight_layout()
plt.savefig(f'../../results/paper-figs/conversion-polymorph-selection.svg', dpi=600,)
plt.show()

## Fig 2d: promote crystallization

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.85))
# ax2 = ax.twinx()
hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)
colors['6A'] = colors['HCP']
num_bins = 13
d0 = '../../results/conversion/betap13.40'
clustB = 'FCC'

for ic, clustA in enumerate(['sp5c', '8A', '6A']): 
    # combine data of all trajectories
    df_ = None
    for i in [2, 3, 6, 9, 11]:
        d1 = f'{d0}/trial{i}'
        df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}.log') # 
        df = df[:125]
        sum = df.sum(axis=0)
        if df_ is None:
            df_ = sum
        else:
            df_ += sum

    # initialize arrays
    conversions_ = np.nan*np.zeros(num_bins)
    source_ = np.nan*np.zeros(num_bins)

    # for cB in ['FCC', 'HCP']:
    for j in range(num_bins):
        # number of particles part of clustA
        source = round(df_[f'number_of_particles_{j}'])
        conversions = round(df_[f'converted_{j}'])
        # converted_ = round(df_[f'{cB}_{j}'])
        divisor = source

        conversions_[j] = conversions / divisor if divisor > 0 else np.nan
        # print(source)
        # conversions_[j] = conversions / divisor
        source_[j] = source
    ax.plot(conversions_[:6], marker='o', c=colors[clustA])
    # ax2.plot(source_[:6]/source_[:].sum(), marker='o', c=colors[clustA])

    ax.set_ylabel(r'$P ~$' + '(conversion)')
    ax.set_xlabel(f'Clusters per particle')
    ax.set_ylim(0, 0.1)
    # ax2.set_ylim(0, 0.2)

    ax.set_xticks(range(6))
    ax.minorticks_on()
    ax.tick_params(axis='x', which='minor', bottom=False)
        
plt.tight_layout()
# plt.savefig(f'../../results/paper-figs/conversion-{clustA}.png', dpi=600,)
plt.savefig(f'../../results/paper-figs/conversion-{clustA}.svg', dpi=600,)

## SI Fluid

### Frequency of OH, PB, SD

In [ ]:
fig, ax = plt.subplots(figsize=(3, 1.85))
hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)
colors['6A'] = colors['HCP']
num_bins = 13
d0 = '../../results/conversion/betap13.40'
clustB = 'FCC'

for ic, clustA in enumerate(['sp5c', '8A', '6A']): 
    # combine data of all trajectories
    df_ = None
    for i in [2, 3, 6, 9, 11]:
        d1 = f'{d0}/trial{i}'
        df = pd.read_csv(f'{d1}/tcc-particle-conversion-{clustA}-{clustB}.log') # 
        df = df[:125]
        sum = df.sum(axis=0)
        if df_ is None:
            df_ = sum
        else:
            df_ += sum

    # initialize arrays
    source_ = np.nan*np.zeros(num_bins)

    # for cB in ['FCC', 'HCP']:
    for j in range(num_bins):
        source = round(df_[f'number_of_particles_{j}'])
        source_[j] = source
    ax.plot(source_[:6]/source_[:].sum(), marker='o', c=colors[clustA])
    print(clustA, source_[1:].sum())

ax.set_ylabel('Frequency')
ax.set_xlabel(f'Clusters per particle')

ax.set_xticks(range(6))
ax.minorticks_on()
ax.tick_params(axis='x', which='minor', bottom=False)

ax.set_ylim(0, None)
        
plt.tight_layout()
# plt.savefig(f'../../results/paper-figs/conversion-{clustA}.png', dpi=600,)
plt.savefig(f'../../results/paper-figs/fluid-dist-{clustA}.svg', dpi=600,)

### Correlation between SD, PB

In [ ]:


for clustA, clustB in [(['8A'], ['FCC']), (['sp5c'], ['8A']), (['sp5c'], ['6A']), ]:
# for clustA, clustB in [(['8A'], ['sp5c']), ]:
    fig, ax = plt.subplots(figsize=(3, 1.85))
    ax2 = ax.twinx()
    hfont = {'family':'sans-serif', 'size': 8}
    mpl.rc('font',family='sans-serif', size=8)
    colors['8A'] = colors['HCP']
    num_bins = 13
    d0 = '../../results/conversion/betap13.40'

    # clustA = ['sp5c']
    # clustB = ['HCP']

    # combine data of all trajectories
    df_ = None
    print(clustA, clustB)
    trajs = [2,]
    for i in trajs:
        d1 = f'{d0}/trial{i}'
        df = pd.read_csv(f'{d1}/tcc-particle-cluster-competition-{clustA}-{clustB}_.log') # 
        df = df[:125]
        sum = df.mean(axis=0)
        if df_ is None:
            df_ = sum / len(trajs)
        else:
            df_ += sum / len(trajs)

    # initialize arrays
    source_ = np.nan*np.zeros(num_bins)
    avg_ = np.nan*np.zeros(num_bins)

    # for cB in ['FCC', 'HCP']:
    for j in range(num_bins):
        source = df_[f'number_of_particles_{j}']
        source_[j] = source
        avg_[j] = df_[f'clusters_per_particle_{j}']

    # ax.plot(source_[:6]/source_[:].sum(), marker='o', c=colors[clustA[0]])
    # print(clustA, source_[1:].sum())
    ax.plot(avg_[:])
    # ax2.plot(source_[:8], c='C1')
    ax2.plot(source_ / source_.sum(), c='C1')

    # ax.set_ylabel('Frequency')
    # ax.set_xlabel(f'Clusters per particle')

    # ax.set_xticks(range(6))
    # ax.minorticks_on()
    # ax.tick_params(axis='x', which='minor', bottom=False)

    # ax.set_ylim(0, None)
            
    plt.tight_layout()
    plt.savefig(f'../../results/paper-figs/fluid-correlation-{clustA}.svg', dpi=600,)
    plt.show()

# Output clusters for OVITO

In [ ]:
def polygon(i, size=1, symmetry=5, phi0=0):
    phi = phi0 + i / symmetry * 2*m.pi
    x = size * m.cos(phi)
    y = size * m.sin(phi)
    return x, y

df = []
sym = 4
plot_spindle = False
plot_8a = False
plot_fcc = True
plot_hcp = False
if plot_fcc or plot_hcp:
    sym = 6
plot_extra_fcc = False
plot_8a_in_fcc_hcp = False

# polygon
start = 1 if plot_8a else 0
start = sym if plot_hcp or plot_fcc else start
for i in range(start, sym):
    size = 0.9 if sym < 5 else 1.0
    x, y = polygon(i, size=size, symmetry=sym)
    if sym == 4:
        df.append(('a', x, y, 0))
    else:
        df.append(('a', x, y, 0))
# spindle
if plot_spindle:
    for i in range(2):
        if plot_8a:
            df.append(('b',-0.1,0,0.6*(-1)**i))
        else:
            df.append(('b',0,0,0.6*(-1)**i))
if plot_8a:
    # shifted
    x, y = polygon(5, symmetry=sym)
    for i in range(2):
        df.append(('b',x,y,0.6*(-1)**i))

if plot_fcc:
    # central particle
    label = 'b' if plot_8a_in_fcc_hcp else 'a'
    df.append((label,0,0,0))
    # hexagonal ring
    for i in range(sym):
        size = 0.7 if sym < 5 else 1.0
        if plot_8a_in_fcc_hcp:
            label = ['b', 'c', 'c', 'a', 'c', 'c'][i]
        else:
            label = 'a'
        x, y = polygon(i, size=size, symmetry=sym)
        df.append((label,x,y,0))
    for i in range(3):
        x, y = polygon(i, symmetry=3, size=0.6, phi0=m.pi/6)
        if plot_8a_in_fcc_hcp:
            label = ['c', 'c', 'b'][i]
        else:
            label = 'b'
        df.append((label,x,y,0.8))
        print(label, x,y)
        if i == 2 and plot_8a_in_fcc_hcp:
            df.append(('b',x+1.02,y,0.8))
    for i in range(3):
        x, y = polygon(i, symmetry=3, size=0.6, phi0=3*m.pi/6)
        if plot_8a_in_fcc_hcp:
            label = ['a', 'a', 'c'][i]
        else:
            label = 'b'
        df.append((label,x,y,-0.8))
        # print(x,y)
        # if i == 1:
        #     df.append(('b',x,y+1.8,0.7))

if plot_hcp:
    # central particle
    label = 'b' if plot_8a_in_fcc_hcp else 'a'
    df.append((label,0,0,0))
    # hexagonal ring
    for i in range(sym):
        size = 0.7 if sym < 5 else 1.0
        if plot_8a_in_fcc_hcp:
            label = ['b', 'a', 'a', 'a', 'c', 'c'][i]
        else:
            label = 'a'
        x, y = polygon(i, size=size, symmetry=sym)
        print(label, x,y)
        df.append((label,x,y,0))
    for i in range(3):
        x, y = polygon(i, symmetry=3, size=0.6, phi0=m.pi/6)
        if plot_8a_in_fcc_hcp:
            label = ['c', 'c', 'b'][i]
        else:
            label = 'b'
        df.append((label,x,y,0.8))
        if i == 2 and plot_8a_in_fcc_hcp:
            df.append(('b',x+1.02,y,0.8))
    for i in range(3):
        x, y = polygon(i, symmetry=3, size=0.6, phi0=m.pi/6)
        label = 'a' if plot_8a_in_fcc_hcp else 'b'
        df.append((label,x,y,-0.8))
    
# more fcc
if plot_extra_fcc:
    for k in np.arange(10):
        for z in np.arange(-8, 6):
            if (k-z) % 2 == 0:
                df.append(('c',x+1.2*k,y,1.2*z+0.6))
            else:
                df.append(('d',x+1.2*k,y,1.2*z+0.6))

if plot_8a:
    file_path = '../../results/paper-figs/clusters/8a.xyz'
elif not (plot_hcp or plot_fcc):
    if sym == 5:
        file_path = '../../results/paper-figs/clusters/sp5c.xyz'
    elif sym == 4:
        file_path = '../../results/paper-figs/clusters/sp4c.xyz'
elif plot_hcp:
    file_path = '../../results/paper-figs/clusters/hcp.xyz'
elif plot_fcc:
    file_path = '../../results/paper-figs/clusters/fcc.xyz'

with open(file_path, 'w') as f:
     f.write(f'{len(df)}\n # comment\n')
df = pd.DataFrame(df)
df.to_csv(file_path, index=False, header=False, mode='a', sep=' ')

## FCC vs HCP in metastable fluid

In [ ]:
d0 = '../../results/conversion/betap13.40'
num_clusters = [None, None]

for ic, clustA in enumerate(['FCC', 'HCP']): 
    # combine data of all trajectories
    df_ = None
    for i in [2, 3, 6, 9, 11]:
        d1 = f'{d0}/trial{i}'
        df = pd.read_csv(f'{d1}/tcc-basic-{clustA}-unfiltered.log')
        if df_ is None:
            df_ = df
        else:
            df_ += df

    print(df)

    num_clusters[ic] = df[:len(df)//2]['number_of_clusters'].sum()

print(num_clusters[0] / (num_clusters[0] + num_clusters[1]))

## Umbrella sampling and FCC vs HCP in experiments

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap
from scipy.ndimage import gaussian_filter


fig, ax = plt.subplots(figsize=(3.3, 3.0))
sigma = 2
beta = 5

myExtent = 205
data = np.loadtxt('../../results/damme/mapNew/2DBarrierFixed_w4_ub10Stitched.txt')
plot = 'n-x' # 'nfcc-nhcp' # # 'n-x'

# Calculating the output and storing it in the array Z
Z = np.zeros((myExtent,myExtent))

for i in range(myExtent):
    for j in range(myExtent):
        Z[i,j] = np.nan

for i in range(len(data)):
    if((data[i,0]<myExtent)and(data[i,1]<myExtent)):
        Z[int(data[i,1]),int(data[i,0])] = data[i,2]

Z2 = np.nan * np.zeros((myExtent,myExtent))
for i in range(myExtent):
    for j in range(myExtent):
        x = j / myExtent
        # x = myExtent * j / (j+1)
        try:
            Z2[i, j] = Z[int(x*i), int((1-x)*i)]
        except:
            continue  

# if plot == 'n-x':
#     Z = Z2

hfont = {'family':'sans-serif', 'size': 8}
mpl.rc('font',family='sans-serif', size=8)

N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(35/N, 255/N, N)
vals[:, 1] = np.linspace(48/N, 197/N, N)
vals[:, 2] = np.linspace(116/N, 227/N, N)
newcmp = ListedColormap(vals)

x = np.array([0,50,150,200])
y = np.array([0,50,150,200])

ax.xaxis.set_ticks(np.arange(0, 201, 50))
ax.yaxis.set_ticks(np.arange(0, 201, 50))
if plot == 'n-x':
    ax.set_xlabel(r'$n_{fcc} + n_{hcp}$', **hfont)
    ax.set_ylabel(r'$n_{fcc} ~/~ (n_{fcc} + n_{hcp})$', **hfont)
else:
    ax.set_xlabel(r'$n_{fcc}$', **hfont)
    ax.set_ylabel(r'$n_{hcp}$', **hfont)

cmap = plt.cm.coolwarm  #'viridis' # #plt.cm.BrBG #plt.cm.coolwarm
for iy in range(myExtent):
    try:
        indices = np.arange(myExtent)
        not_nan = ~np.isnan(Z[:, iy])
        # Z[:, iy] = np.interp(indices, indices[not_nan], Z[:, iy][not_nan])

        not_nan = ~np.isnan(Z2[:, iy])
        Z2[:, iy] = np.interp(indices, indices[not_nan], Z2[:, iy][not_nan])
    except:
        continue

Z_ = gaussian_filter(Z, sigma=sigma, mode='nearest')
Z2_ = gaussian_filter(Z2, sigma=sigma, mode='nearest')

cmin = 0 if plot == 'n-x' else -32 #-20
cmax = 32 if plot == 'n-x' else 40 # 30
dx = 1 if plot == 'n-x' else 2 #1
bounds = np.arange(cmin, cmax+dx, dx) #if plot == 'n-x' else None
norm = mpl.colors.BoundaryNorm(boundaries=bounds, ncolors=256) #if plot == 'n-x' else None
Z_plot = np.flip(Z2_.T, axis=0) if plot == 'n-x' else np.floor(Z)
im = ax.imshow(
    Z_plot,
    cmap=cmap, origin='lower', interpolation='none',
    norm=norm, aspect='auto', clim=(cmin, cmax)) #    


i, f = 40, 200
# Z2_ = np.flip(Z2_.T, axis=0)
Z3 = np.flip(Z2_, axis=-1)[:, i:f]
# expected composition
comp = np.sum(np.arange(i, f) * np.exp(-beta*Z3), axis=-1) / np.sum(np.exp(-beta*Z3), axis=-1)
comp = np.arange(i, f)[np.argmin(Z3, axis=-1)]
if plot == 'n-x':
    plt.plot(np.arange(myExtent)[:], comp[:], ls='--', c='k')
else:
    pass
    x = comp / myExtent
    n = np.arange(myExtent)
    plt.plot(n*x, n*(1-x), ls='--', c='k', lw=1)

# Nc
Nc = np.sum(np.arange(myExtent)[:, None] * np.exp(beta*Z3), axis=-2) / np.sum(np.exp(beta*Z3), axis=-2)
Nc = np.arange(myExtent)[:, None][np.argmax(Z3, axis=-2)]

if plot == 'n-x':
    plt.plot(Nc, np.arange(i, f), ls='-', c='k')
else:
    plt.plot(np.arange(i, f), Nc-np.arange(i, f), ls='-', c='k', lw=1)
    pass



if True:
    for frame_number in range(1, 23):
        savedir = '../../results/experiments/'+f'dump-bops/{frame_number}'
        with open(savedir + '/nuclei.pickle', 'rb') as file:
            nuclei = pickle.load(file)
            for nucleus in nuclei:
                Nx, nfcc, nhcp = nucleus
                if Nx > 20 and Nx < 200:
                    if plot == 'n-x':
                        plt.scatter(Nx, nfcc/Nx*myExtent, marker='.', s=5, c='k', alpha=1)
                    else:
                        plt.scatter(nfcc, nhcp, marker='.', s=1, c='k', alpha=1)

plt.xlim(0, 200)
labels = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9] if plot == 'n-x' else None# [60, 80, 100, 120, 140, 160, 180]
ticks = [myExtent * x for x in labels] if plot == 'n-x' else None #[60, 80, 100, 120, 140, 160, 180]
plt.yticks(ticks, labels=labels)

if plot == 'n-x':
    plt.ylim(myExtent*0.3, 190)
else:
    plt.ylim(0, 200)
cbar = plt.colorbar(im) #shrink=0.75, aspect=20*0.75
# if plot == 'n-x':
#     cbar.ax.set_yticks([0, 2, 4, 6, 8, 10, 12, 14, 16])  # vertically oriented colorbar
cbar.ax.set_title(r'$\beta \Delta G$')
plt.tight_layout()
plt.savefig(f'../../results/paper-figs/Fig1-softmax.png', bbox_inches='tight', dpi=600)
# plt.savefig(f'../../results/paper-figs/us-{plot}.svg', bbox_inches='tight', dpi=900)
plt.show()


In [ ]:
# Gc = np.sum(Z3 * np.exp(Z3), axis=-2) / np.sum(np.exp(Z3), axis=-2)
# plt.plot(Gc)
# print(Gc.min(), Gc.max())
print(Z3.shape)
plt.plot(Z2[140])

In [ ]:
### GABRIELE's CODE ###
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import ListedColormap

myExtent = 205
dividingFactor = 2
data = np.loadtxt('../../results/damme/mapNew/2DBarrierFixed_w4_ub10Stitched.txt')

# Calculating the output and storing it in the array Z
Z = np.zeros((myExtent,myExtent))

for i in range(myExtent):
    for j in range(myExtent):
        Z[i,j] = np.nan

for i in range(len(data)):
    #print(i,data[i])
    if((data[i,0]<myExtent)and(data[i,1]<myExtent)):
        Z[int(data[i,1]),int(data[i,0])] = int(data[i,2]/dividingFactor)
        

fig, ax = plt.subplots(figsize=(4,3))
hfont = {'family':'sans-serif','fontname':'Helvetica', 'size': 8}
mpl.rc('font', family='sans-serif', size=8)

N = 256
vals = np.ones((N, 4))
vals[:, 0] = np.linspace(35/N, 255/N, N)
vals[:, 1] = np.linspace(48/N, 197/N, N)
vals[:, 2] = np.linspace(116/N, 227/N, N)
newcmp = ListedColormap(vals)

x = np.array([0,50,150,200])
y = np.array([0,50,150,200])

ax.xaxis.set_ticks(np.arange(0, 201, 50))
ax.yaxis.set_ticks(np.arange(0, 201, 50))
ax.set_xlabel(r'$n_{fcc}$', **hfont)
ax.set_ylabel(r'$n_{hcp}$', **hfont)

cmap = plt.cm.coolwarm
im = ax.imshow(Z, cmap=cmap, origin='lower',interpolation='none', clim=(-20, 20)) #

### MY CODE
for frame_number in range(1, 23):
    savedir = '../../results/experiments/'+f'dump-bops/{frame_number}'
    with open(savedir + '/nuclei.pickle', 'rb') as file:
        nuclei = pickle.load(file)
        for nucleus in nuclei:
            Nx, nfcc, nhcp = nucleus
            if Nx > 40 and Nx < 400:
                plt.scatter(nfcc, nhcp, marker='.', s=5, c='k', alpha=1)

plt.xlim(0, 200)
plt.ylim(0, 200)
plt.tight_layout()
fig.colorbar(im)
plt.savefig('../../results/paper-figs/cmap.png', bbox_inches='tight', dpi=600, format='png')

In [ ]:
# print(Z2_[100, :])
# plt.plot(logsumexp(Z2_, b=np.arange(myExtent), axis=-1))

Z3 = np.flip(Z2_, axis=-1)[:, 60:180]
# plt.plot(np.arange(myExtent), (60+np.argmin(Z3, axis=-1)) / myExtent)
comp = np.sum(np.arange(60, 180) * np.exp(-Z3), axis=-1) / np.sum(np.exp(-Z3), axis=-1)
# plt.plot(np.arange(myExtent), comp/myExtent)

print(Z3.shape)
Nc = np.sum(np.arange(myExtent)[:, None] * np.exp(Z3), axis=-2) / np.sum(np.exp(Z3), axis=-2)
plt.plot(np.arange(60, 180), Nc)

# plt.plot(np.arange(myExtent), (logsumexp(-Z3, b=np.arange(60, 180), axis=-1))/myExtent)
print(Z3.shape)

print(60/myExtent)

## Coexistence clusters vs distance

In [ ]:
dists = {}
for plane in ['fcc-110', 'hcp-1120']: #['fcc-100', 'fcc-110', 'hcp-1120', 'hcp-1010', 'hcp-0001']:
    dists[plane] = {}
    for c in full_clusterlist:
        logdir = d0 = f'../../results/coexistence_md/{plane}'
        mode = 'particles-in-cluster' #
        # mode = 'clusters-per-particle'
        logfile = f'{logdir}/dump/tcc-{c}-spatial-distribution-{mode}.log'
        dists[plane][c] = pd.read_csv(logfile).values
        dists[plane][c] = dists[plane][c][:].mean(axis=0)[1:]

In [ ]:
fig, axs = plt.subplots(8, 4, sharex=True, sharey='row', figsize=(7,8)) #(8.27,11.69)
axs = np.concatenate(axs)

plt.subplots_adjust(hspace=0.1)
plt.subplots_adjust(wspace=0.1)
w = 20

clusters_to_plot = included_clusterlist
mean_pops = []

for plane in ['fcc-110', 'hcp-1120']: #['fcc-100', 'fcc-110', 'hcp-1120', 'hcp-1010', 'hcp-0001'][1:2]:
    for ic, clust in enumerate(clusters_to_plot):
        color = colors['FCC'] if plane.startswith('fcc') else colors['HCP']
        label = 'FCC' if plane.startswith('fcc') else 'HCP'
        label = label if ic == 0 else None
        Lx = 32.26 if label == 'FCC' else 32.58
        x = 32.5 * np.arange(len(dists[plane][clust])) / len(dists[plane][clust])
        axs[ic].plot(32.5-x, dists[plane][clust], label=label, c=color)
        label = labels[clust] if clust in labels else clust
        y = 0.8 if clust in ['sp4a', 'sp5a'] else 0.1
        weight = 'bold' if clust in ['8A', '6A', 'sp5c', 'FCC', 'HCP'] else None
        axs[ic].text(0.1, y, label, transform=axs[ic].transAxes, weight=weight, size=8)
        axs[ic].fill_between([11, 18], 0, 2, color='#e2e2e2')
        axs[ic].set_xlim(0, 30)
        axs[ic].set_ylim(0, 1.05)
        axs[ic].set_yticks([0, 0.5, 1.0])

fig.supylabel('P (particle in cluster)', **hfont)

fig.supxlabel(r'$x ~/~ \sigma$', **hfont)
fig.legend(bbox_to_anchor=(0.4, 1, 0.25, 0.2), loc="lower left", ncol=2, mode="expand", fontsize=10)
fig.tight_layout()
plt.savefig(f"../../results/paper-figs/SI_all_clusters_distance.pdf", dpi=300, bbox_inches='tight')